In [15]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image



In [16]:
import torch
from torch.utils.data import Dataset
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image

class FaceLandmarksDataset(Dataset):
    def __init__(
        self,
        data_path: str = r'D:\vhproj\dt_landmarks\data_image\ibug_300W_large_face_landmark_dataset\labels_ibug_300W_train.xml',
        transform=None,
        target_size=(224, 224),
        bbox_scale_factor=1.2,
        apply_augmentation=False
    ):
        # parse XML
        tree = ET.parse(data_path)
        root = tree.getroot()

        self.image_filenames = []
        self.landmarks = []
        self.crops = []
        self.transform = transform
        self.root_dir = r'D:\vhproj\dt_landmarks\data_image\ibug_300W_large_face_landmark_dataset'
        self.target_size = target_size
        self.bbox_scale_factor = bbox_scale_factor
        self.apply_augmentation = apply_augmentation

        # đọc thông tin file và landmarks
        for filename in root[2]:
            img_path = os.path.join(self.root_dir, filename.attrib['file'])
            self.image_filenames.append(img_path)
            self.crops.append(filename[0].attrib)

            lm = []
            for num in range(68):
                x = int(filename[0][num].attrib['x'])
                y = int(filename[0][num].attrib['y'])
                lm.append([x, y])
            self.landmarks.append(lm)

        self.landmarks = np.array(self.landmarks, dtype=np.float32)
        assert len(self.image_filenames) == len(self.landmarks)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        # đọc ảnh, nếu lỗi thì next
        while True:
            image = cv2.imread(self.image_filenames[index])
            if image is None:
                index = (index + 1) % len(self.image_filenames)
                continue
            break

        # chuyển về grayscale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        landmarks = self.landmarks[index]
        bbox = self.crops[index]

        # tính crop theo bbox và scale factor
        x1 = int(bbox['left'])
        y1 = int(bbox['top'])
        w = int(bbox['width'])
        h = int(bbox['height'])
        new_w = int(w * self.bbox_scale_factor)
        new_h = int(h * self.bbox_scale_factor)
        x1 = max(0, x1 - (new_w - w) // 2)
        y1 = max(0, y1 - (new_h - h) // 2)
        x2 = min(image.shape[1], x1 + new_w)
        y2 = min(image.shape[0], y1 + new_h)
        cropped = image[y1:y2, x1:x2]

        # scale landmarks vào crop
        lm_scaled = landmarks - np.array([x1, y1], dtype=np.float32)
        # resize
        resized = cv2.resize(cropped, self.target_size)
        sx = self.target_size[1] / cropped.shape[1]
        sy = self.target_size[0] / cropped.shape[0]
        lm_resized = np.stack([lm_scaled[:,0] * sx, lm_scaled[:,1] * sy], axis=1).astype(np.float32)

        # augmentation (nếu có)
        if self.apply_augmentation:
            pil_img = Image.fromarray(resized)
            orig_w, orig_h = pil_img.size

            if np.random.rand() > 0.5:
                pil_img = pil_img.transpose(Image.FLIP_LEFT_RIGHT)
                lm_resized[:,0] = orig_w - lm_resized[:,0]

            angle = np.random.uniform(-30, 30)
            pil_img = pil_img.rotate(angle)
            rot_mat = cv2.getRotationMatrix2D((orig_w/2, orig_h/2), angle, 1.0)
            ones = np.ones((lm_resized.shape[0], 1), dtype=np.float32)
            lm_aug = np.hstack([lm_resized, ones])
            lm_resized = (rot_mat @ lm_aug.T).T

            resized = np.array(pil_img)

        # transform ảnh
        if self.transform:
            resized = self.transform(resized)
        else:
            resized = torch.tensor(resized, dtype=torch.float32).unsqueeze(0) / 255.0

        # flatten landmarks thành 1D tensor
        landmarks_tensor = torch.tensor(lm_resized.flatten(), dtype=torch.float32)

        return resized, landmarks_tensor


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torchvision.models import resnet18

class FacialLandmarksModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super(FacialLandmarksModel, self).__init__()
        self.learning_rate = learning_rate
        
        self.backbone = resnet18(pretrained=True)
        
        
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.backbone.fc = nn.Linear(512, 136)
        
        self.criterion = nn.MSELoss()
        # Lưu lại hàm Loss
        self.loss_value = []

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        images, landmarks = batch
        predictions = self(images)
        loss = self.criterion(predictions, landmarks)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, landmarks = batch
        predictions = self(images)
        val_loss = self.criterion(predictions, landmarks)
        self.log('val_loss', val_loss)
        self.loss_value.append(val_loss)
        return val_loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [18]:
dataset = FaceLandmarksDataset()
# Tính kích thước
dataset_size = len(dataset)
train_size = int(0.6 * dataset_size)
val_size   = int(0.2 * dataset_size)
test_size  = dataset_size - train_size - val_size

# Chia dataset
train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Tạo DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# In ra số phần tử cho kiểm tra
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


Train: 3999, Val: 1333, Test: 1334


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import os

# Tạo thư mục lưu model nếu chưa có
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Callback: Lưu model tốt nhất
checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir,
    filename="best_model",
    save_top_k=1,
    monitor="val_loss",
    mode="min"
)

# Callback: Dừng sớm nếu không cải thiện
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    patience=10,
    verbose=True,
    mode="min"
)

# Khởi tạo model
model = FacialLandmarksModel(learning_rate=1e-3)

# Khởi tạo trainer
trainer = Trainer(
    max_epochs=100,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    callbacks=[checkpoint_callback, early_stop_callback],
    log_every_n_steps=10
)

# Train model
trainer.fit(model, train_loader, val_loader)


c:\Users\firek\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\firek\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | backbone  | ResNet  | 11.2 M | train
1 | criterion | MSELoss | 0      | train
-----------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\firek\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


In [ ]:
import matplotlib.pyplot as plt

# Hàm vẽ loss sau khi train
def plot_loss(model):
    # Chuyển loss_value sang CPU nếu nó đang ở trên GPU
    loss_values = [loss.cpu().item() for loss in model.loss_value]
    plt.plot(loss_values, label='Validation Loss')
    plt.title('Loss per Validation Step')
    plt.xlabel('Validation Step')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Sau khi huấn luyện
plot_loss(model_)


In [ ]:
import matplotlib.pyplot as plt

# Hàm vẽ loss sau khi train từ iterate thứ 100
def plot_loss(model, start_iter=100):
    # Chuyển loss_value sang CPU nếu nó đang ở trên GPU
    loss_values = [loss.cpu().item() for loss in model.loss_value]
    
    # Lấy loss từ iterate thứ 100 trở đi
    loss_values = loss_values[start_iter:]
    
    # Vẽ biểu đồ
    plt.plot(loss_values, label='Validation Loss')
    plt.title(f'Loss per Validation Step (from step {start_iter})')
    plt.xlabel('Validation Step')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Sau khi huấn luyện
plot_loss(model_, start_iter=100)


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load model đã huấn luyện từ file checkpoint

model.eval()  # Đặt model ở chế độ eval để test
dataset = FaceLandmarksDataset('/kaggle/input/ibug-300w/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml')
# Chuẩn bị transform cho ảnh test
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Chọn một ảnh từ dataset
index =  104  # Bạn có thể thay đổi index để chọn ảnh khác
image_path = dataset.image_filenames[index]

# Đọc ảnh từ file
image = cv2.imread(image_path)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Lấy bounding box và landmarks từ dataset
bbox = dataset.crops[index]
landmarks = dataset.landmarks[index]

# Crop ảnh theo bounding box
x1 = int(bbox['left'])
y1 = int(bbox['top'])
x2 = int(bbox['left']) + int(bbox['width'])
y2 = int(bbox['top']) + int(bbox['height'])

cropped_image = image_gray[y1:y2, x1:x2]

# Resize ảnh về kích thước đầu vào của mô hình
resized_image = cv2.resize(cropped_image, (224, 224))

# Chuyển đổi ảnh thành tensor
input_image = torch.tensor(resized_image, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0  # Thêm batch size và kênh

# Dự đoán landmarks
with torch.no_grad():
    predicted_landmarks = model(input_image)

# Đưa các điểm dự đoán về kích thước ban đầu
predicted_landmarks = predicted_landmarks.view(-1, 2).numpy()
scale_x = cropped_image.shape[1] / 224.0
scale_y = cropped_image.shape[0] / 224.0
predicted_landmarks[:, 0] *= scale_x
predicted_landmarks[:, 1] *= scale_y

# Vẽ các điểm landmarks lên ảnh
for (x, y) in predicted_landmarks:
    cv2.circle(cropped_image, (int(x), int(y)), 2, (255, 0, 0), -1)

# Hiển thị ảnh và các điểm landmarks
plt.imshow(cropped_image, cmap='gray')
plt.show()


In [ ]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Tạo DataLoader cho tập test
test_dataset = FaceLandmarksDataset(data_path='/kaggle/input/ibug-300w/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml', 
                                    transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Load model đã huấn luyện từ file checkpoint
model.eval()  # Chuyển model sang chế độ eval để dự đoán

# Tính tổng loss và số lượng mẫu
total_mse = 0.0
total_samples = 0

# Vòng lặp qua tập test để dự đoán và tính MSE
with torch.no_grad():
    for batch in test_loader:
        images, true_landmarks = batch
        predicted_landmarks = model(images)

        # Tính MSE cho batch hiện tại
        mse = F.mse_loss(predicted_landmarks, true_landmarks, reduction='sum')  # Tổng MSE cho tất cả landmarks
        total_mse += mse.item()  # Thêm MSE của batch hiện tại vào tổng MSE
        total_samples += images.size(0)  # Cộng số lượng mẫu vào tổng số mẫu

# Tính MSE trung bình
average_mse = total_mse / total_samples

print(f'Mean Squared Error (MSE) trên tập test: {average_mse}')


In [ ]:
# Lưu mô hình dưới dạng .pth
torch.save(model.state_dict(), '/kaggle/working/finalpth.pth')


In [ ]:
# Lưu mô hình dưới dạng .ckpt (sử dụng ModelCheckpoint)
trainer.save_checkpoint('/kaggle/working/finalckpt.ckpt')


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load model đã huấn luyện từ file checkpoint
model = FacialLandmarksModel.load_from_checkpoint('/kaggle/working/facial_landmarks_model_11_10_1.ckpt')
model.eval()  # Đặt model ở chế độ eval để test

# Chuẩn bị transform cho ảnh test
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Chọn một ảnh từ dataset
index = 901# Bạn có thể thay đổi index để chọn ảnh khác
image_path = dataset.image_filenames[index]

# Đọc ảnh từ file
image = cv2.imread(image_path)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Lấy bounding box và landmarks từ dataset
bbox = dataset.crops[index]
landmarks = dataset.landmarks[index]

# Crop ảnh theo bounding box
x1 = int(bbox['left'])
y1 = int(bbox['top'])
x2 = int(bbox['left']) + int(bbox['width'])
y2 = int(bbox['top']) + int(bbox['height'])

cropped_image = image_gray[y1:y2, x1:x2]

# Resize ảnh về kích thước đầu vào của mô hình
resized_image = cv2.resize(cropped_image, (224, 224))

# Chuyển đổi ảnh thành tensor
input_image = torch.tensor(resized_image, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0  # Thêm batch size và kênh

# Dự đoán landmarks
with torch.no_grad():
    predicted_landmarks = model(input_image)

# Đưa các điểm dự đoán về kích thước ban đầu
predicted_landmarks = predicted_landmarks.view(-1, 2).numpy()
scale_x = cropped_image.shape[1] / 224.0
scale_y = cropped_image.shape[0] / 224.0
predicted_landmarks[:, 0] *= scale_x
predicted_landmarks[:, 1] *= scale_y

# Vẽ các điểm landmarks lên ảnh
for (x, y) in predicted_landmarks:
    cv2.circle(cropped_image, (int(x), int(y)), 2, (255, 0, 0), -1)

# Hiển thị ảnh và các điểm landmarks
plt.imshow(cropped_image, cmap='gray')
plt.show()


In [ ]:
# Tạo DataLoader cho tập test
test_dataset = FaceLandmarksDataset(data_path='/kaggle/input/ibug-300w/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml', 
                                    transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load model đã huấn luyện từ file checkpoint
model.eval()  # Chuyển model sang chế độ eval để dự đoán

# Tính tổng loss và số lượng mẫu
total_mse = 0.0
total_samples = 0

# Số lượng điểm mốc
number_of_landmarks = true_landmarks.size(1)  # Giả định true_landmarks đã được định nghĩa

# Vòng lặp qua tập test để dự đoán và tính MSE
with torch.no_grad():
    for batch in test_loader:
        images, true_landmarks = batch
        predicted_landmarks = model(images)

        # Tính MSE cho từng ảnh trong batch
        mse_per_image = F.mse_loss(predicted_landmarks, true_landmarks, reduction='none')  # Tính MSE cho từng ảnh
        mse_per_image = mse_per_image.view(mse_per_image.size(0), -1)  # Đưa về dạng (batch_size, number_of_landmarks * 2)
        
        # Tính MSE trung bình cho từng ảnh
        mse_average_per_image = mse_per_image.sum(dim=1) / number_of_landmarks  # Tính MSE trung bình cho mỗi ảnh
        
        total_mse += mse_average_per_image.sum().item()  # Cộng dồn tổng MSE trung bình cho từng ảnh
        total_samples += images.size(0)  # Cộng số lượng mẫu vào tổng số mẫu

# Tính MSE trung bình trên toàn bộ tập test
mean_mse_per_image = total_mse / total_samples

print(f'MSE trung bình trên mỗi ảnh: {mean_mse_per_image}')


In [ ]:
# Load model đã huấn luyện từ file checkpoint
model.eval()  # Chuyển model sang chế độ eval để dự đoán

# Danh sách để lưu trữ MSE cho từng ảnh
mse_per_image_list = []

# Số lượng điểm mốc
number_of_landmarks = true_landmarks.size(1)  # Giả định true_landmarks đã được định nghĩa

# Vòng lặp qua tập test để dự đoán và tính MSE
with torch.no_grad():
    for idx in range(len(test_dataset)):
        # Lấy từng ảnh và nhãn từ test_dataset
        image, true_landmarks = test_dataset[idx]  # Lấy ảnh và nhãn
        image = image.unsqueeze(0)  # Thêm chiều batch vào ảnh

        predicted_landmarks = model(image)  # Dự đoán landmarks

        # Tính MSE cho ảnh hiện tại
        mse = F.mse_loss(predicted_landmarks, true_landmarks.unsqueeze(0), reduction='mean')  # Tính MSE cho từng ảnh

        mse_per_image_list.append(mse.item())  # Thêm MSE vào danh sách

# In MSE cho từng ảnh
for idx, mse in enumerate(mse_per_image_list):
    print(f'Ảnh {idx + 1}: MSE = {mse}')
